In [2]:
import sys
sys.path.append('/opt/ctpn/CTPN/tools/')
from cfg import Config as cfg
from other import enlarge_boxes, get_output_name, draw_boxes, resize_im, CaffeModel
import cv2, os, caffe, sys
from detectors import TextProposalDetector, TextDetector
import os.path as osp
from utils.timer import Timer
from IPython.core.display import Image, display
from tesseract_ocr import tesseract_ocr 


DEMO_IMAGE_DIR="demo_images/pic_folder"
NET_DEF_FILE="models/deploy.prototxt"
MODEL_FILE="models/ctpn_trained_model.caffemodel"

caffe.set_mode_cpu()

#if len(sys.argv)>1 and sys.argv[1]=="--no-gpu":
#    caffe.set_mode_cpu()
#else:
#    caffe.set_mode_gpu()
#    caffe.set_device(cfg.TEST_GPU_ID)

text_proposals_detector=TextProposalDetector(CaffeModel(NET_DEF_FILE, MODEL_FILE))
text_detector=TextDetector(text_proposals_detector)

In [3]:
!rm images/* && instagram-scraper --location 894641523 -m 25 --media_types image -q -d images

rm: cannot remove 'images/*': No such file or directory


In [ ]:
from glob import glob
timer=Timer()
for im_name in glob("demo_images/*.jpg"):
    print im_name
    im_file=osp.join(im_name)
    im=cv2.imread(im_file)
    timer.tic()
    im, f=resize_im(im, cfg.SCALE, cfg.MAX_SCALE)
    text_lines,text_proposals,scores=text_detector.detect(im)
#    text_lines=enlarge_boxes(text_lines)
    print "Time: %f"%timer.toc()
    print text_lines
    im_with_text_lines=draw_boxes(im, text_lines, caption=im_name, is_display=True)
    box_count = 0
    for box in text_lines:
        left, top, right, bottom = box[:4]
        img_height, img_width = im.shape[0], im.shape[1]
        padding_x = int((right - left) / 4)
        width = padding_x * 2 + (right - left)
        height = (64 * width / 128)
        padding_y = int((height - (bottom - top))/2)
        new_left = int(max(0, left - padding_x))
        new_right = int(min(img_width - 1, right + padding_x))
        new_top = int(max(0, top - padding_y))
        new_bottom = int(min(img_height - 1, bottom + padding_y))

        crop_img = im[new_top: new_bottom, new_left: new_right]
        #crop_img = cv2.cvtColor(crop_img,cv2.COLOR_RGB2GRAY)
        crop_img = cv2.resize(crop_img, (300,300))
        cv2.imwrite('demo_images/pic_folder/'+im_name+'_box_text_proposals_{}.jpg'.format(box_count),crop_img)
        box_count += 1     

demo_images/IMG_20170922_142015.jpg
Time: 29.331481
[[  64.          382.83969116  447.          452.96044922    0.97270793]
 [ 192.          168.49615479  351.          229.11680603    0.88479215]
 [  64.          297.52767944  447.          375.90692139    0.67968184]]
